In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/autoam-car-price-prediction/sample_submission.csv
/kaggle/input/autoam-car-price-prediction/train.csv
/kaggle/input/autoam-car-price-prediction/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/autoam-car-price-prediction/train.csv')
test = pd.read_csv('/kaggle/input/autoam-car-price-prediction/test.csv')

In [3]:
train

,model,year,motor_type,running,wheel,color,type,status,motor_volume,price
0,toyota,2022,petrol,3000 km,left,skyblue,sedan,excellent,2.0,24500
1,mercedes-benz,2014,petrol,132000 km,left,black,sedan,excellent,2.0,25500
2,kia,2018,petrol,95000 miles,left,other,sedan,excellent,2.0,11700
3,mercedes-benz,2002,petrol,137000 miles,left,golden,sedan,excellent,3.2,12000
4,mercedes-benz,2017,petrol,130000 km,left,black,sedan,good,2.0,26000
...,...,...,...,...,...,...,...,...,...,...
1637,hyundai,2017,petrol,120000 miles,left,white,sedan,good,2.0,12400
1638,toyota,2014,petrol,170000 km,left,black,sedan,good,2.0,16500
1639,nissan,2018,petrol,68900 miles,left,blue,suv,good,2.0,19500
1640,nissan,2019,petrol,31000 miles,left,black,suv,excellent,2.0,19500


In [4]:
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import category_encoders as ce

In [5]:
def convert_to_km(value):
    numeric_value, unit = value.split()
    numeric_value = float(numeric_value)
    if unit.lower() == 'miles':
        return numeric_value * 1.60934
    elif unit.lower() == 'km':
        return numeric_value
    else:
        return None

In [6]:
lst_duplicated = list(train[train.duplicated()].index)

In [7]:
train.drop(index=lst_duplicated, inplace=True)

In [8]:
train['running_numeric'] = train['running'].apply(convert_to_km)

In [9]:
train.drop(columns=['wheel', 'running'], inplace=True)

In [10]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [11]:
encoder = ce.TargetEncoder(return_df=True)

In [12]:
train

,model,year,motor_type,color,type,status,motor_volume,price,running_numeric
0,toyota,2022,petrol,skyblue,sedan,excellent,2.0,24500,3000.000
1,mercedes-benz,2014,petrol,black,sedan,excellent,2.0,25500,132000.000
2,kia,2018,petrol,other,sedan,excellent,2.0,11700,152887.300
3,mercedes-benz,2002,petrol,golden,sedan,excellent,3.2,12000,220479.580
4,mercedes-benz,2017,petrol,black,sedan,good,2.0,26000,130000.000
...,...,...,...,...,...,...,...,...,...
1637,hyundai,2017,petrol,white,sedan,good,2.0,12400,193120.800
1638,toyota,2014,petrol,black,sedan,good,2.0,16500,170000.000
1639,nissan,2018,petrol,blue,suv,good,2.0,19500,110883.526
1640,nissan,2019,petrol,black,suv,excellent,2.0,19500,49889.540


In [13]:
train['running_numeric'] = train['running_numeric'].astype('int64')
train['motor_volume'] = train['motor_volume'].astype('int64')

In [14]:
train_y = train['price']
train_X = train.drop(columns=['price'])

In [15]:
X_train_loo = encoder.fit_transform(train_X, train_y)

In [16]:
X_train_loo

,model,year,motor_type,color,type,status,motor_volume,running_numeric
0,17656.814229,2022,16884.113314,16075.404914,15323.695584,17402.645973,2,3000
1,14735.613466,2014,16884.113314,17201.804469,15323.695584,17402.645973,2,132000
2,14828.007067,2018,16884.113314,14536.273603,15323.695584,17402.645973,2,152887
3,14735.613466,2002,16884.113314,14845.206130,15323.695584,17402.645973,3,220479
4,14735.613466,2017,16884.113314,17201.804469,15323.695584,11913.603604,2,130000
...,...,...,...,...,...,...,...,...
1637,14833.040346,2017,16884.113314,16841.155172,15323.695584,11913.603604,2,193120
1638,17656.814229,2014,16884.113314,17201.804469,15323.695584,11913.603604,2,170000
1639,18270.739255,2018,16884.113314,13430.333326,18519.066667,11913.603604,2,110883
1640,18270.739255,2019,16884.113314,17201.804469,18519.066667,17402.645973,2,49889


In [17]:
train_y = train['price']
train_X = X_train_loo

In [18]:
from sklearn.linear_model import LinearRegression

lr=LinearRegression()
lr.fit(train_X,train_y)
print('Training Accuracy of Linear Regression', lr.score(train_X,train_y))

Training Accuracy of Linear Regression 0.43980973543878754


In [19]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1.0)
ridge.fit(train_X,train_y)
print('Training Accuracy of Ridge Regression : ', ridge.score(train_X,train_y))

Training Accuracy of Ridge Regression :  0.4398097039118045


In [20]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=1.0)
lasso.fit(train_X,train_y)
print('Training Accuracy of Lasso Regression : ', lasso.score(train_X,train_y))

Training Accuracy of Lasso Regression :  0.43980957511254115


In [21]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor()
dtr.fit(train_X,train_y)
print('Training Accuracy of Decision Tree Regressor : ', dtr.score(train_X,train_y))

Training Accuracy of Decision Tree Regressor :  0.9977410259361883


In [22]:
from sklearn.ensemble import RandomForestRegressor

model_rfr = RandomForestRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=5,n_estimators = 100)
model_rfr.fit(train_X,train_y)
print('Training Accuracy of Random Forest Regressor : ', model_rfr.score(train_X,train_y))

Training Accuracy of Random Forest Regressor :  0.8369506811292284


In [23]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor()
gbr.fit(train_X,train_y)
print('Training Accuracy of Gradient Boosting Classifier : ', gbr.score(train_X,train_y))

Training Accuracy of Gradient Boosting Classifier :  0.8637032107529399


In [24]:
test['running_numeric'] = test['running'].apply(convert_to_km)

In [25]:
test.drop(columns=['wheel', 'running'], inplace=True)

In [26]:
test['running_numeric'] = test['running_numeric'].astype('int64')
test['motor_volume'] = test['motor_volume'].astype('int64')

In [27]:
test

,Id,model,year,motor_type,color,type,status,motor_volume,running_numeric
0,0,kia,2020,petrol,black,sedan,excellent,2,24000
1,1,nissan,2017,petrol,white,suv,excellent,2,136793
2,2,hyundai,2021,petrol,white,sedan,excellent,2,48280
3,3,kia,2018,petrol,silver,sedan,excellent,2,85295
4,4,mercedes-benz,2003,petrol,black,Universal,normal,1,230000
...,...,...,...,...,...,...,...,...,...
406,406,nissan,2021,petrol,black,suv,excellent,2,33400
407,407,hyundai,2017,petrol,other,sedan,excellent,2,96560
408,408,mercedes-benz,2012,petrol,white,sedan,good,2,218000
409,409,kia,2020,petrol,red,sedan,good,2,64373


In [28]:
X_test_loo = encoder.transform(test.drop(columns=['Id']))

In [29]:
X_test_loo

,model,year,motor_type,color,type,status,motor_volume,running_numeric
0,14828.007067,2020,16884.113314,17201.804469,15323.695584,17402.645973,2,24000
1,18270.739255,2017,16884.113314,16841.155172,18519.066667,17402.645973,2,136793
2,14833.040346,2021,16884.113314,16841.155172,15323.695584,17402.645973,2,48280
3,14828.007067,2018,16884.113314,13561.881823,15323.695584,17402.645973,2,85295
4,14735.613466,2003,16884.113314,17201.804469,17725.020815,8042.238531,1,230000
...,...,...,...,...,...,...,...,...
406,18270.739255,2021,16884.113314,17201.804469,18519.066667,17402.645973,2,33400
407,14833.040346,2017,16884.113314,14536.273603,15323.695584,17402.645973,2,96560
408,14735.613466,2012,16884.113314,16841.155172,15323.695584,11913.603604,2,218000
409,14828.007067,2020,16884.113314,14913.012593,15323.695584,11913.603604,2,64373


In [30]:
Grad = gbr.predict(X_test_loo)

In [31]:
Prediction = pd.DataFrame({"Id" : test["Id"], "Price":Grad})
Prediction.set_index("Id")

,Price
Id,
0,17282.816851
1,17095.701383
2,22720.222301
3,14147.360149
4,13654.367603
...,...
406,27555.099925
407,13905.433273
408,13835.306663


In [32]:
Prediction.to_csv('submission.csv', index = False)